In [198]:
import re
import urllib
import urllib.request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
%matplotlib inline
plt.style.use('ggplot')
sns.set_style('darkgrid')

In [199]:
# embaralhei a lista de players pra nao pegar só de GM - NAO PRECISA RODAR ESSA CEDULA DNV
# shuffled_players.to_csv('shuffled_player_list_chesscom.csv',index = False)

In [200]:
players = pd.read_csv('shuffled_player_list_chesscom.csv')

In [201]:
players

,username,title,bullet_rating,blitz_rating,rapid_rating
0,michaelq2d5,GM,2793,2765,2903
1,bernd_stromberg,CM,2619,2701,0
2,g2_coder,FM,2350,2200,0
3,mennabarreto,FM,2251,2407,0
4,radojeiriz,FM,2388,2271,2489
...,...,...,...,...,...
8556,skiiier,FM,2492,2477,0
8557,volodymyrmolyboha,NM,2431,2503,2189
8558,miodrag-jevtic,FM,2378,2356,1936
8559,jerryle,NM,2214,2251,1365


In [202]:
# resolve erro "This event loop is already running"
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [203]:
# instala e importa https://github.com/sarartur/chess.com
# wrapper para dados publicados pelo chess.com
!pip install chessdotcom
import chessdotcom

ERROR: Could not find a version that satisfies the requirement chessdotcom
ERROR: No matching distribution found for chessdotcom


In [211]:
# peguei um role do stack overflow e editei, pega os anos/meses que teve jogo de cada pessoa

def getDates(user):
    username = user #change 
    baseUrl = "https://api.chess.com/pub/player/" + username + "/games/"
    archivesUrl = baseUrl + "archives"

    #read the archives url and store in a list
    f = urllib.request.urlopen(archivesUrl)
    archives = f.read().decode("utf-8")
    archives = archives.replace("{\"archives\":[\"", "\",\"")
    archivesList = archives.split("\",\"" + baseUrl)
    archivesList[len(archivesList)-1] = archivesList[len(archivesList)-1].rstrip("\"]}")
    archivesList.remove('')
    month = []
    year = []
    for i in archivesList:
        year.append(int(i[0:4]))
        month.append(int(i[5:]))
    return(month, year)

In [205]:
games_db = pd.read_csv('empty_games.csv')
games_db.head()

,id,ranked,time_control,status,winner,white,black,moves


In [207]:
def checkstatus(white_result, black_result):
    if white_result != 'win':
        return white_result
    else:
        return black_result
    
def checkwinner(white, black):
    if white['result'] == 'win':
        return white['username']
    elif black['result'] == 'win':
        return black['username']
    else:
        return 'draw'

def trataPGN(pgn):
    # remove clock
    txt = '{\[%clk \d+:\d+:\d+(\.\d+)?\]}'
    a = re.sub(txt, "", pgn)

    # remove header
    txt = '\[.+\]'
    a = re.sub(txt, "", a)
    a

    # remove \n
    txt = '\\n'
    a = re.sub(txt, "", a)
    a
    
    return(a)    

In [208]:
# escolhe que jogadores voce quer pegar
players_to_db = players[0:3]
players_to_db.head()

,username,title,bullet_rating,blitz_rating,rapid_rating
0,michaelq2d5,GM,2793,2765,2903
1,bernd_stromberg,CM,2619,2701,0
2,g2_coder,FM,2350,2200,0


In [212]:
games_db = pd.read_csv('empty_games.csv')

for k in players_to_db['username']:
    month, year = getDates(k)
    for i in range(len(month)):
        response = chessdotcom.get_player_games_by_month(k,month=month[i],year=year[i]).json
        for j in (response['games']):
            try:
                temp = {
                    'id':j['uuid'],
                    'ranked':j['rated'],
                    'time_control': j['time_class'],
                    'status': checkstatus(j['white']['result'],j['black']['result']),
                    'winner': checkwinner(j['white'],j['black']),
                    'white': (j['white']['username']),
                    'black': (j['black']['username']),
                    'moves': trataPGN(j['pgn'])
                }
                #print((j['white']['result'],j['black']['result']))
                games_db = games_db.append(temp,ignore_index=True)
            except:
                temp = 0

In [213]:
games_db

,id,ranked,time_control,status,winner,white,black,moves
0,db338032-0f2b-11df-8000-000000010001,True,blitz,resigned,michaelq2d5,michaelq2d5,malachi76,1. d4 1... d5 2. c4 2... c6 3. cxd5 3... ...
1,38c7cc80-0f2c-11df-8000-000000010001,True,blitz,checkmated,malachi76,malachi76,michaelq2d5,1. e4 1... e5 2. Nf3 2... Nc6 3. Bc4 3......
2,978d4682-0f2c-11df-8000-000000010001,True,blitz,resigned,michaelq2d5,Jojo_Dy,michaelq2d5,1. e4 1... e5 2. Nf3 2... Nc6 3. Bb5 3......
3,ca373ea8-0f2c-11df-8000-000000010001,True,blitz,timeout,Surfsmurf,michaelq2d5,Surfsmurf,1. d4 1... d6 2. Nf3 2... Bg4 3. c4 3... ...
4,3e71e200-0f2d-11df-8000-000000010001,True,blitz,resigned,michaelq2d5,Adolf_Anderssen,michaelq2d5,1. e4 1... e5 2. Nf3 2... Nc6 3. Bb5 3......
...,...,...,...,...,...,...,...,...
36443,52dfcc80-aa6b-11e6-8000-000000010001,True,bullet,resigned,SaudiSalman,SaudiSalman,G2_Coder,1. e3 1... e5 2. Bd3 2... Qf6 3. f4 3... ...
36444,a5b9843c-aa6b-11e6-8000-000000010001,True,bullet,checkmated,G2_Coder,ricfazolim,G2_Coder,1. e4 1... e5 2. Nf3 2... Bc5 3. c3 3... ...
36445,2037e1f8-ab08-11e6-8000-000000010001,True,blitz,abandoned,G2_Coder,Vin_it007,G2_Coder,1. Nf3 1... Nf6 2. d4 2... e5 3. c3 3... ...
36446,08fdf1a8-ab12-11e6-8000-000000010001,True,blitz,checkmated,G2_Coder,G2_Coder,10101987,1. d4 1... e6 2. Nf3 2... Qf6 3. e4 3... ...


In [214]:
# salva em csv
games_db.to_csv('games_db_1_3.csv',index = False)